In [53]:
import pandas as pd
from bertopic import BERTopic
from transformers import pipeline

import re
from bs4 import BeautifulSoup

In [54]:
df = pd.read_csv("reddit.csv")

/tmp/ipython-input-95727286.py:1: DtypeWarning: Columns (1,2,4,8,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("reddit.csv")


In [55]:
# Create a summary DataFrame
missing_summary = pd.DataFrame({
    'missing_count': df.isnull().sum(),
    'missing_percentage': (df.isnull().sum() / len(df)) * 100
})

# Sort by missing percentage (descending)
missing_summary = missing_summary.sort_values('missing_percentage', ascending=False)
print(missing_summary)

                 missing_count  missing_percentage
Unnamed: 12             165260           99.999395
title                   154754           93.642178
num_comments            154745           93.636732
id                      154738           93.632496
bank_name                50556           30.591610
sentiment_score          10575            6.398969
sentiment                10575            6.398969
parent_post_id           10569            6.395338
comment_id               10527            6.369924
text                       125            0.075638
subreddit                   65            0.039332
score                       64            0.038727
permalink                   62            0.037516
query                       62            0.037516
kind                        55            0.033281
created_utc                 44            0.026625
join_key                    19            0.011497


In [56]:
texts = df['text'].dropna().tolist()
texts_clean = [str(text).lower().strip() for text in texts]

print(f"Loaded {len(texts_clean)} documents")
print(f"First text: {texts_clean[0][:200]}...")

test_run = texts_clean[:1100]

Loaded 165136 documents
First text: this is entirely curiosity. i am not currently in this situation and simply want to know how the inner workings of wire transfers work, i am not complaining about the process!! (caveat because i have ...


In [57]:
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

cleaned_docs = []
for doc in test_run:
    # Remove punctuation and convert to lowercase
    doc = doc.translate(str.maketrans('', '', string.punctuation)).lower()
    words = doc.split()
    filtered = [word for word in words if word not in stop_words]
    cleaned_docs.append(' '.join(filtered))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
topic_model = BERTopic(embedding_model = "all-MiniLM-L6-v2")

In [59]:
topics, probs = topic_model.fit_transform(cleaned_docs)

In [60]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,382,-1_account_bank_money_check,"[account, bank, money, check, would, im, credi...",[peculiar situation thats happened years ago o...
1,0,155,0_wire_bank_transfer_number,"[wire, bank, transfer, number, sent, money, us...",[hi would appreciate input anyone experience i...
2,1,123,1_savings_account_iâ_im,"[savings, account, iâ, im, looking, one, money...",[hi everyone iâ€™m 18 currently freshman commu...
3,2,78,2_cd_rate_interest_apy,"[cd, rate, interest, apy, rates, month, cds, m...",[ive read basically advantage cd locked rate l...
4,3,71,3_bank_card_account_got,"[bank, card, account, got, back, iâ, phone, mo...",[tldr three pending charges draftkingcom pulle...
5,4,49,4_overdraft_fee_account_pay,"[overdraft, fee, account, pay, bank, iâ, charg...",[citizens checking account pending overdraft f...
6,5,37,5_check_checks_deposit_account,"[check, checks, deposit, account, mobile, bank...",[mistakes made month financially along misunde...
7,6,35,6_chase_account_iâ_branch,"[chase, account, iâ, branch, bank, said, call,...",[hi donâ€™t know right help would greatly appr...
8,7,32,7_work_teller_banking_job,"[work, teller, banking, job, iâ, interview, li...",[hello 23m graduated college degree business m...
9,8,30,8_mom_account_bank_address,"[mom, account, bank, address, estate, money, m...",[posting case anyone else dealt something simi...


In [63]:
import pandas as pd

# Create a table with document-topic assignments
doc_topics_df = pd.DataFrame({
    'Document': cleaned_docs,
    'Topic': topics,
    'Top_Keywords': [', '.join([word for word, _ in topic_model.get_topic(t)[:5]])
                     if t != -1 else 'Outlier' for t in topics]
})

print(doc_topics_df.head(10))

                                            Document  Topic  \
0  entirely curiosity currently situation simply ...      0   
1  iâ€™m tired understaffed work busier bank 2 te...      7   
2  im college student student account pnc told cl...     13   
3  hi iâ€™m sure post figured would start weeks a...      8   
4  case background iâ€™m freelancer south east as...      0   
5  capone changes thinking changing banks everyon...     -1   
6  today third time receive call customer service...      7   
7  iâ€™m start interviewing jp morgan private cli...      7   
8  bank decided freeze checking credit card accou...      3   
9  11 months ago spouse moved shared apartment li...     -1   

                             Top_Keywords  
0      wire, bank, transfer, number, sent  
1          work, teller, banking, job, iâ  
2  pnc, account, business, accounts, told  
3     mom, account, bank, address, estate  
4      wire, bank, transfer, number, sent  
5                                 Outlier 